# Optimización de Procesos en Spark

Spark proporciona varias técnicas de optimización que permiten mejorar la eficiencia del procesamiento distribuido:

- Catalyst Optimizer: motor de optimización para operaciones SQL y DataFrames.
- Tungsten: optimización a nivel de memoria y CPU.
- Particiones y persistencia: control de ejecución y uso eficiente de recursos.

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

spark = SparkSession.builder.appName("PySpark07").getOrCreate()

In [ ]:
# Datos de ejemplo
data = [
    ("Alice", "HR", 3000),
    ("Bob", "IT", 4500),
    ("Cathy", "HR", 3200),
    ("David", "IT", 5000),
    ("Eve", "Finance", 4000),
    ("Frank", "Finance", 4200)
]
columns = ["name", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.createOrReplaceTempView("employees")
df.show()

In [ ]:
# Consulta optimizada automáticamente
query = spark.sql("""
    SELECT department, AVG(salary) 
    FROM employees 
    WHERE salary > 3000 
    GROUP BY department
""")
query.explain()

## Cache y persistencia

Spark permite mantener en memoria los resultados intermedios para evitar su recálculo.

In [ ]:
df_filtered = df.filter(df.salary > 3000)
df_filtered.cache()
df_filtered.count()  # Primera acción: carga en caché
df_filtered.show()   # Segunda acción: uso desde caché

## Particiones

Usando Spark, podemos particionar los datos por una o más columnas. Cuando particionamos un conjunto de datos, lo estamos dividiendo en varios archivos. Por lo tanto, podemos leer solo una partición relevante cuando nos interese, y no todos los datos.
Un ejemplo útil de dividir nuestros datos sería por la columna 'año', o las columnas 'año' y 'mes', si corresponde. De esta manera, podríamos acceder a un conjunto de datos correspondientes a una fecha específica.

- `repartition(n)`: redistribuye los datos con `n` particiones (shuffle).
- `coalesce(n)`: reduce el número de particiones sin shuffle (eficiente).
- `partitionBy(column)`: redistribuye los datos sesgún los posibles valores de una columna.

In [ ]:
# Define el schema
schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("views", IntegerType(), True),
    StructField("quality", StringType(), True)
])

# Leer archivo CSV
movies = spark.read.option("header", True).schema(schema).csv("../../data/movies.csv")

In [ ]:
# Guardar resultados en particiones
movies.write.partitionBy("quality").parquet("data/partition_by_quality.parquet")
# Leer una de las particiones guardadas
medium_quality_data = spark.read.parquet("data/partition_by_quality.parquet/quality=medium")
medium_quality_data.show()

### Formato Parquet
Un archivo Parquet es un formato de archivo columnar utilizado para almacenar datos de una manera eficiente y optimizada. En lugar de guardar los datos en filas (como un archivo CSV o una tabla tradicional), Parquet los organiza por columnas, lo que permite consultas y análisis más rápidos, especialmente cuando se manejan grandes volúmenes de datos.

## Buenas prácticas

✅ Usar DataFrames en vez de RDDs para aprovechar Catalyst.  
✅ Evitar `collect()` en grandes volúmenes.  
✅ Revisar el plan de ejecución con `.explain()`.  
✅ Usar `cache()` o `persist()` si reutilizas resultados.  
✅ Ajustar particiones en base al volumen de datos y núcleos disponibles.

## Ejercicios

1. Leemos el dataset '../../data/movies.csv' y contamos el número de particiones.
2. Reparticionar a 8 particiones, realizar una transformación y cachear el resultado.
3. Usar `.explain()` para analizar el plan de ejecución antes y después del caché.
4. Aplicar una función Window (lab04) para calcular el top 3 de películas según visualizaciones para cada tipo de calidad 

#### Apartado 1

In [ ]:
# Cargar datos
df = spark.read.option("header", True).csv("../../data/movies.csv")
df.show(5)
print('Número de particiones inciales:', df.rdd.getNumPartitions())

#### Apartado 2

In [ ]:
partitions = df.repartition(8)
filtered = partitions.filter('views > 10000')
filtered.cache()

#### Apartado 3

Antes de filtrar y guardar en caché:

In [ ]:
partitions.explain()

¿Qué está pasando?
- Lectura directa del CSV: No hay filtros ni transformaciones. Spark está leyendo el archivo tal cual.
- Reparticionado (`RoundRobin 8`): Los datos se redistribuyen equitativamente entre 8 particiones.
- `AdaptiveSparkPlan isFinalPlan=false`: Spark ha planificado adaptativamente, pero aún no ha ejecutado la consulta, por lo que el plan podría cambiar en tiempo de ejecución (AQE en acción).
- Sin caché: Cada vez que ejecutes una acción sobre este DataFrame, Spark leerá nuevamente desde el disco.

Después de aplicar filtros y guardar en caché:

In [ ]:
filtered.explain()

¿Qué significa?
- Filtro aplicado:
    - Spark filtra los registros `Filter (isnotnull(views#416) AND (cast(views#416 as int) > 10000))`.
- Reparticionado después del filtro: Se realiza una redistribución de datos en 8 particiones después de aplicar el filtro.
- `InMemoryRelation`:
    - El DataFrame fue cacheado, así que ahora está en memoria (deserializado) y disponible con respaldo en disco si hace falta.
- `InMemoryTableScan`:
    - Las acciones siguientes a la caché ya no leerán desde el CSV, sino directamente desde memoria. Esto mejora significativamente el rendimiento.

#### Apartado 4

In [ ]:
# Aplicar función de ventana: Top 3 películas por calidad según las visualizaciones
window_spec = Window.partitionBy("quality").orderBy(col("views").desc())
df_with_rank = df.withColumn("rank", row_number().over(window_spec))

df_top3 = df_with_rank.filter(col("rank") <= 3)
df_top3.show()

In [37]:
spark.stop()